In [0]:
%run "/Workspace/Repos/ashwin.nair8089@gmail.com/inshorts/library/connector"

In [0]:
%run "/Workspace/Repos/ashwin.nair8089@gmail.com/inshorts/library/password_manager"

In [0]:
conn, success, message = snowflake_connector("ashwin009", get_password('snf_conn'), "WSYCJNH-JF51761",'COMPUTE_WH','INSHORTS','ETL')

In [0]:
cursor = conn.cursor()

In [0]:

cursor.execute('Truncate table foundation.retention_summary')
print(cursor.fetchall())
cursor.execute(f'''INSERT INTO foundation.retention_summary
SELECT 
    u.install_dt, 
	COUNT(DISTINCT u.deviceid),
    COUNT(DISTINCT CASE WHEN e.event_date = DATEADD(DAY, 1, u.install_dt) THEN e.deviceid END) AS D1_retained,
    COUNT(DISTINCT CASE WHEN e.event_date = DATEADD(DAY, 2, u.install_dt) THEN e.deviceid END) AS D2_retained,
    COUNT(DISTINCT CASE WHEN e.event_date BETWEEN u.install_dt AND DATEADD(DAY, 7, u.install_dt) THEN e.deviceid END) AS W1_retained,
    COUNT(DISTINCT CASE WHEN e.event_date BETWEEN u.install_dt AND DATEADD(MONTH, 1, u.install_dt) THEN e.deviceid END) AS M1_retained,
    COUNT(DISTINCT CASE WHEN e.event_date BETWEEN u.install_dt AND DATEADD(MONTH, 3, u.install_dt) THEN e.deviceid END) AS M3_retained
FROM ETL.user u
LEFT JOIN (
    SELECT deviceid, DATE(eventtimestamp) AS event_date
    FROM ETL.event
) e ON u.deviceid = e.deviceid
GROUP BY u.install_dt;''')
print(cursor.fetchall())